In [151]:
# python packages
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import tensorflow.keras
import cv2
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import joblib
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout, SeparableConv2D, MaxPool2D
from tensorflow.keras.models import Sequential, Model
import tensorflow.keras
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from sklearn import datasets, metrics

In [152]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [153]:
IMAGE_SIZE=[150,150]

In [154]:
datagen = ImageDataGenerator(rescale = 1./255, 
                             shear_range= 0.2, 
                             horizontal_flip= True, 
                             validation_split= 0.2,
                             dtype=tensorflow.float32)


training_set = datagen.flow_from_directory('/content/drive/MyDrive/datasets', 
                                           
                                           target_size=IMAGE_SIZE, 
                                           batch_size=64, 
                                           class_mode='binary',
                                           subset='training',
                                           shuffle=True, 
                                           seed=123, )

test_set = datagen.flow_from_directory('/content/drive/MyDrive/datasets',
                                       target_size=IMAGE_SIZE,
                                       batch_size=64, 
                                       class_mode='binary', 
                                       subset='validation', 
                                       shuffle=True, 
                                       seed=123, )

Found 1981 images belonging to 2 classes.
Found 494 images belonging to 2 classes.


In [155]:
from keras.callbacks import EarlyStopping
from keras.layers import Flatten, Dense, Conv2D, AveragePooling2D, BatchNormalization, SpatialDropout2D, MaxPooling2D
from keras.utils import to_categorical
from pathlib import Path

In [156]:
nb_classes = 2

In [157]:
model = Sequential()


model.add(Conv2D(64, (3,3), activation='relu', input_shape=IMAGE_SIZE + [3],padding='same',))
model.add(Conv2D(64, (3,3), activation='relu', padding='same',))
tensorflow.keras.layers.MaxPool2D(),

model.add(Conv2D(128, (3,3), activation='relu', padding='same',))
model.add(Conv2D(128, (3,3), activation='relu', padding='same',))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(256, (3,3), activation='relu', padding='same',))
model.add(Conv2D(256, (3,3), activation='relu', padding='same',))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

#model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.3))

model.add(Dense(1, activation = 'sigmoid'))

In [158]:
sgd = tensorflow.keras.optimizers.SGD(learning_rate=0.01)

In [159]:
initial_learning_rate = 0.01
def lr_exp_decay(epoch, lr):
    k = 0.1
    return initial_learning_rate * math.exp(-k*epoch)

In [160]:
model.compile(loss = 'binary_crossentropy', optimizer= sgd, metrics = ['accuracy'])

In [161]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_92 (Conv2D)           (None, 150, 150, 64)      1792      
_________________________________________________________________
conv2d_93 (Conv2D)           (None, 150, 150, 64)      36928     
_________________________________________________________________
conv2d_94 (Conv2D)           (None, 150, 150, 128)     73856     
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 150, 150, 128)     147584    
_________________________________________________________________
batch_normalization_34 (Batc (None, 150, 150, 128)     512       
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 75, 75, 128)       0         
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 75, 75, 256)     

In [162]:
'''model= Sequential()

model.add(Conv2D(64, kernel_size= (3,3), activation='tanh', input_shape=IMAGE_SIZE + [3],padding='same',strides=(1,1)))
model.add(AveragePooling2D(pool_size=(2,2), strides=(2,2), padding= 'valid'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Conv2D(64, kernel_size= (3,3), activation= 'relu', input_shape= IMAGE_SIZE + [3], padding='same',
                strides= (1,1)))
model.add(AveragePooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(BatchNormalization())

model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer= 'adam', metrics = ['accuracy'])'''

"model= Sequential()\n\nmodel.add(Conv2D(64, kernel_size= (3,3), activation='tanh', input_shape=IMAGE_SIZE + [3],padding='same',strides=(1,1)))\nmodel.add(AveragePooling2D(pool_size=(2,2), strides=(2,2), padding= 'valid'))\nmodel.add(BatchNormalization())\nmodel.add(Dropout(0.5))\n\nmodel.add(Conv2D(64, kernel_size= (3,3), activation= 'relu', input_shape= IMAGE_SIZE + [3], padding='same',\n                strides= (1,1)))\nmodel.add(AveragePooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))\nmodel.add(BatchNormalization())\n\nmodel.add(Dropout(0.3))\n\nmodel.add(Flatten())\nmodel.add(Dropout(0.3))\nmodel.add(Dense(128, activation='relu'))\nmodel.add(Dropout(0.3))\nmodel.add(Dense(64, activation='relu'))\nmodel.add(Dropout(0.3))\nmodel.add(Dense(32, activation='relu'))\nmodel.add(Dropout(0.3))\nmodel.add(Dense(1, activation = 'sigmoid'))\n\nmodel.compile(loss = 'binary_crossentropy', optimizer= 'adam', metrics = ['accuracy'])"

In [163]:
def lr_scheduler(epoch, lr):
    decay_rate = 0.1
    decay_step = 90
    if epoch % decay_step == 0 and epoch:
        return lr * decay_rate
    return lr

callbacks = [
    tensorflow.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)
]

In [165]:
history1 = model.fit(training_set,epochs=34,steps_per_epoch=10,
                    validation_steps=7,validation_data=test_set, callbacks=callbacks,shuffle=True)

Epoch 1/34

Epoch 00001: LearningRateScheduler reducing learning rate to 0.009999999776482582.
10/10 [==============================] - 29s 1s/step - loss: 6.8901 - accuracy: 0.5197 - val_loss: 0.7512 - val_accuracy: 0.5000
Epoch 2/34

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009999999776482582.
10/10 [==============================] - 14s 1s/step - loss: 2.4573 - accuracy: 0.4926 - val_loss: 0.6779 - val_accuracy: 0.5379
Epoch 3/34

Epoch 00003: LearningRateScheduler reducing learning rate to 0.009999999776482582.
10/10 [==============================] - 27s 3s/step - loss: 1.2679 - accuracy: 0.5464 - val_loss: 0.6868 - val_accuracy: 0.4978
Epoch 4/34

Epoch 00004: LearningRateScheduler reducing learning rate to 0.009999999776482582.
10/10 [==============================] - 14s 1s/step - loss: 0.8894 - accuracy: 0.5338 - val_loss: 0.6724 - val_accuracy: 0.5179
Epoch 5/34

Epoch 00005: LearningRateScheduler reducing learning rate to 0.009999999776482582.
10/10 [==

KeyboardInterrupt: ignored

In [ ]:
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

model.save_weights('model_weights.h5')

In [ ]:
import numpy as np
from keras.preprocessing import image
from keras.models import model_from_json
from pathlib import Path

class_label = ["COVID-19 CASE", "NON COVID-19 CASE"]

#test_image= image.load_img('/content/drive/MyDrive/FinalYearProject/datasets/CT_NonCOVID/1078.png', target_size = (64,64))
#Wrong-test_image= image.load_img('/content/drive/MyDrive/FinalYearProject/datasets/CT_NonCOVID/1859.png', target_size = (64,64))
#test_image= image.load_img('/content/drive/MyDrive/FinalYearProject/datasets/CT_NonCOVID/887.png', target_size = (64,64))
#test_image= image.load_img('/content/drive/MyDrive/FinalYearProject/datasets/CT_NonCOVID/3.jpg', target_size = (64,64))
#test_image= image.load_img('/content/drive/MyDrive/FinalYearProject/datasets/CT_COVID/2020.03.18.20038125-p16-56-2.png', target_size = (64,64))

f = Path("model_structure.json")
model_structure = f.read_text()

model = model_from_json(model_structure)

model.load_weights("model_weights.h5")

uploaded_img = image.load_img('/content/drive/MyDrive/heart image scan-300x225.jpg')
img= image.load_img('/content/drive/MyDrive/heart image scan-300x225.jpg', target_size = IMAGE_SIZE+ [3])

test_image = image.img_to_array(img)/255

listOfImages = np.expand_dims(test_image, axis = 0)

result = model.predict(listOfImages)

single_result = result[0]

mostLikelyClass = int(np.argmax(single_result))

class_likelihood = single_result[mostLikelyClass]

class_label = class_label[mostLikelyClass]
print("image is a {} - likelihood: {:2f}".format(class_label, class_likelihood))


#classes = training_set.class_indices

#print(training_set.class_indices)
if result[0][0] >= 0.7 :
  prediction_Model = 'PREDICTION: Non Covid (Healthy)'
elif result[0][0] <= 0.05:
  prediction_Model = 'PREDICTION: Non Covid (Healthy) or scan is irelevant'
else:
  prediction_Model = 'PREDICTION: Covid Case'
print('\n\n')
print(prediction_Model)
plt.title('UPLOADED IMAGE')
plt.imshow(uploaded_img)
plt.xlabel('Uploaded CT scan image')

In [ ]:
print(history1.history.keys())
# summarize history for accuracy
plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])
plt.title('model3 accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('model3 loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()